In [1]:
import pandas as pd
from TwitterUser import TwitterUser
from NetworkController import NetworkController
import tweepy
from tokens import *
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
def apiv1_login():
    auth = tweepy.OAuth1UserHandler(consumer_key_public, consumer_key_private, access_token_public, access_token_private)
    api_v1 = tweepy.API(auth, wait_on_rate_limit=True)
    return api_v1

api = apiv1_login()

In [6]:
df = pd.read_csv('us_senators.csv')
# df = df.loc[(df.followers_count>5000) & (df.friends_count > 30)]
# df = df.sample(100, random_state=20522001).reset_index()
df

,State,Portrait,Senator,Party,Party.1,Born,Occupation(s),Previous elective office(s),Education,Assumed office,Class,Residence[1],id,name,screen_name,location,followers_count,friends_count
0,Alabama,NaN,Tommy Tuberville,NaN,Republican,(age 68),College football coach Investment management f...,None,Southern Arkansas University (BS),"January 3, 2021",2026 Class 2,Auburn[2],110798061,Tommy Tuberville,TTuberville,"Alabama, USA",77306,745
1,Alabama,NaN,Katie Britt,NaN,Republican,(age 41),Senate staffer University administrator Lawyer...,None,"University of Alabama (BS, JD)","January 3, 2023",2028 Class 3,Montgomery,1400915708259696640,Katie Britt for AL,KatieBrittforAL,NaN,21965,2
2,Alaska,NaN,Lisa Murkowski,NaN,Republican,(age 65),Lawyer,Alaska House of Representatives,Georgetown University (AB) Willamette Universi...,"December 20, 2002[c]",2028 Class 3,Girdwood,151247615,Lisa Murkowski,LisaForSenate,Alaska,9238,479
3,Alaska,NaN,Dan Sullivan,NaN,Republican,(age 58),U.S. Marine Corps officer Lawyer Assistant Sec...,Alaska Attorney General,Harvard University (AB) Georgetown University ...,"January 3, 2015",2026 Class 2,Anchorage,1869975300,Dan Sullivan,DanSullivan_AK,"Anchorage, AK",4928,577
4,Arizona,NaN,Kyrsten Sinema,NaN,Independent[d],(age 46),Social worker Political activist Lawyer Colleg...,U.S. House Arizona Senate Arizona House of Rep...,Brigham Young University (BA) Arizona State Un...,"January 3, 2019",2024 Class 1,Phoenix,1080844782,Kyrsten Sinema,SenatorSinema,NaN,200953,724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,West Virginia,NaN,Shelley Moore Capito,NaN,Republican,(age 69),College career counselor West Virginia Board o...,U.S. House West Virginia House of Delegates,Duke University (BA) University of Virginia (MEd),"January 3, 2015",2026 Class 2,Charleston,193794406,Shelley Moore Capito,SenCapito,NaN,93394,1089
96,Wisconsin,NaN,Ron Johnson,NaN,Republican,(age 67),Accountant Corporate executive,None,University of Minnesota (BS),"January 3, 2011",2028 Class 3,Oshkosh,142332083,Ron Johnson,RonJohnsonWI,"Oshkosh, Wisconsin",57260,2714
97,Wisconsin,NaN,Tammy Baldwin,NaN,Democratic,(age 60),Lawyer,"U.S. House Wisconsin Assembly Dane County, Wis...",Smith College (BA) University of Wisconsin-Mad...,"January 3, 2013",2024 Class 1,Madison,87510313,Tammy Baldwin,tammybaldwin,Wisconsin,52829,4119
98,Wyoming,NaN,John Barrasso,NaN,Republican,(age 70),Orthopedic surgeon Medical chief of staff Nonp...,Wyoming Senate,Rensselaer Polytechnic Institute Georgetown Un...,"June 25, 2007[ad]",2024 Class 1,Casper,499268312,John Barrasso,barrassoforwyo,"Casper, Wyoming",1235,39


In [17]:
x = api.get_friendship(source_id=df.iloc[0].id.item(), target_id=df.iloc[2].id.item())



State                                                                     Alaska
Portrait                                                                     NaN
Senator                                                           Lisa Murkowski
Party                                                                        NaN
Party.1                                                               Republican
Born                                                                    (age 65)
Occupation(s)                                                             Lawyer
Previous elective office(s)                      Alaska House of Representatives
Education                      Georgetown University (AB) Willamette Universi...
Assumed office                                              December 20, 2002[c]
Class                                                               2028 Class 3
Residence[1]                                                            Girdwood
id                          

In [22]:
# y = api.lookup_friendships(s)

Friendship(_api=<tweepy.api.API object at 0x000002535B9B47F0>, _json={'id': 1006419421244678144, 'id_str': '1006419421244678144', 'screen_name': 'GretaThunberg', 'following': False, 'followed_by': False, 'following_received': None, 'following_requested': None}, id=1006419421244678144, id_str='1006419421244678144', screen_name='GretaThunberg', following=False, followed_by=False, following_received=None, following_requested=None)

In [20]:
# Create a dataframe to store the relationships or if already done just read it

try:
    relationships = pd.read_csv('relationships.csv')
except:
    relationships = pd.DataFrame(columns=['person1', 'person2', 'relationship'])
    
# Iterate over all pairs of users
for i, person1 in enumerate(df["name"]):
    for j, person2 in enumerate(df["name"]):
        if i >= j or not relationships.loc[(relationships.person1 == person1) & (relationships.person2 == person2)].empty:
            continue
        
        try:
            relationship = api.get_friendship(source_screen_name=df.loc[i, "screen_name"], 
                                               target_screen_name=df.loc[j, "screen_name"])
            
            # Check if person1 follows person2
            if relationship[0].following:
                # Check if person2 follows person1
                if relationship[1].following:
                    relationships = relationships.append({"person1": person1, "person2": person2, 
                                                         "relationship": "following"}, ignore_index=True)
                else:
                    relationships = relationships.append({"person1": person1, "person2": person2, 
                                                         "relationship": "following only by person1"}, ignore_index=True)
            else:
                # Check if person2 follows person1
                if relationship[1].following:
                    relationships = relationships.append({"person1": person1, "person2": person2, 
                                                         "relationship": "following only by person2"}, ignore_index=True)
                else:
                    relationships = relationships.append({"person1": person1, "person2": person2, 
                                                         "relationship": "not following"}, ignore_index=True)
            print(relationships)
        except Exception as e:
            print(f"Error: {e}")
        time.sleep(1837.144/500)
        
relationships.to_csv('relationships.csv', index=False)

            person1             person2               relationship
0  Tommy Tuberville  Katie Britt for AL  following only by person1
            person1             person2               relationship
0  Tommy Tuberville  Katie Britt for AL  following only by person1
1  Tommy Tuberville      Lisa Murkowski              not following
            person1             person2               relationship
0  Tommy Tuberville  Katie Britt for AL  following only by person1
1  Tommy Tuberville      Lisa Murkowski              not following
2  Tommy Tuberville        Dan Sullivan              not following
            person1             person2               relationship
0  Tommy Tuberville  Katie Britt for AL  following only by person1
1  Tommy Tuberville      Lisa Murkowski              not following
2  Tommy Tuberville        Dan Sullivan              not following
3  Tommy Tuberville      Kyrsten Sinema              not following
            person1             person2               relation

Rate limit reached. Sleeping for: 459


             person1             person2               relationship
0   Tommy Tuberville  Katie Britt for AL  following only by person1
1   Tommy Tuberville      Lisa Murkowski              not following
2   Tommy Tuberville        Dan Sullivan              not following
3   Tommy Tuberville      Kyrsten Sinema              not following
4   Tommy Tuberville  Captain Mark Kelly              not following
..               ...                 ...                        ...
59  Tommy Tuberville     Martin Heinrich              not following
60  Tommy Tuberville       Ben Ray Luján              not following
61  Tommy Tuberville       Chuck Schumer              not following
62  Tommy Tuberville  Kirsten Gillibrand              not following
63  Tommy Tuberville         Thom Tillis              not following

[64 rows x 3 columns]
             person1             person2               relationship
0   Tommy Tuberville  Katie Britt for AL  following only by person1
1   Tommy Tuberville     

Rate limit reached. Sleeping for: 544


                person1                 person2               relationship
0      Tommy Tuberville      Katie Britt for AL  following only by person1
1      Tommy Tuberville          Lisa Murkowski              not following
2      Tommy Tuberville            Dan Sullivan              not following
3      Tommy Tuberville          Kyrsten Sinema              not following
4      Tommy Tuberville      Captain Mark Kelly              not following
..                  ...                     ...                        ...
147  Katie Britt for AL              Jon Tester              not following
148  Katie Britt for AL            Steve Daines              not following
149  Katie Britt for AL     Senator Deb Fischer              not following
150  Katie Britt for AL      Gov. Pete Ricketts              not following
151  Katie Britt for AL  Catherine Cortez Masto              not following

[152 rows x 3 columns]
                person1                 person2               relationship
0

...

               person1              person2               relationship
0     Tommy Tuberville   Katie Britt for AL  following only by person1
1     Tommy Tuberville       Lisa Murkowski              not following
2     Tommy Tuberville         Dan Sullivan              not following
3     Tommy Tuberville       Kyrsten Sinema              not following
4     Tommy Tuberville   Captain Mark Kelly              not following
...                ...                  ...                        ...
1636        Jon Ossoff           Todd Young              not following
1637        Jon Ossoff           Mike Braun              not following
1638        Jon Ossoff       Chuck Grassley              not following
1639        Jon Ossoff           Joni Ernst              not following
1640        Jon Ossoff  Senator Jerry Moran              not following

[1641 rows x 3 columns]
               person1              person2               relationship
0     Tommy Tuberville   Katie Britt for AL  followi

Rate limit reached. Sleeping for: 176


               person1                        person2  \
0     Tommy Tuberville             Katie Britt for AL   
1     Tommy Tuberville                 Lisa Murkowski   
2     Tommy Tuberville                   Dan Sullivan   
3     Tommy Tuberville                 Kyrsten Sinema   
4     Tommy Tuberville             Captain Mark Kelly   
...                ...                            ...   
1816      Brian Schatz  U.S. Senator Cindy Hyde-Smith   
1817      Brian Schatz                    Josh Hawley   
1818      Brian Schatz                   Eric Schmitt   
1819      Brian Schatz                     Jon Tester   
1820      Brian Schatz                   Steve Daines   

                   relationship  
0     following only by person1  
1                 not following  
2                 not following  
3                 not following  
4                 not following  
...                         ...  
1816              not following  
1817              not following  
1818     

Rate limit reached. Sleeping for: 176


               person1             person2               relationship
0     Tommy Tuberville  Katie Britt for AL  following only by person1
1     Tommy Tuberville      Lisa Murkowski              not following
2     Tommy Tuberville        Dan Sullivan              not following
3     Tommy Tuberville      Kyrsten Sinema              not following
4     Tommy Tuberville  Captain Mark Kelly              not following
...                ...                 ...                        ...
1996        Mike Crapo           Ron Wyden              not following
1997        Mike Crapo        Jeff Merkley              not following
1998        Mike Crapo       Bob Casey Jr.              not following
1999        Mike Crapo      John Fetterman              not following
2000        Mike Crapo   Senator Jack Reed              not following

[2001 rows x 3 columns]
               person1             person2               relationship
0     Tommy Tuberville  Katie Britt for AL  following only by per

In [65]:
relationships.to_csv('relationships.csv', index=False)

In [43]:
not relationships.loc[(relationships.person1 == person1) & (relationships.person2 == person2)].empty

True